<a href="https://colab.research.google.com/github/anne-urai/ddm_mediation/blob/main/HDDMnn_mediation_simulations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# First, set Runtime -> Runtime type -> GPU

# https://hddm.readthedocs.io/en/latest/lan_tutorial.html
!pip install scikit-learn
!pip install cython
!pip install pymc

!pip install -U --no-deps git+https://github.com/hddm-devs/hddm
!pip install -U --no-deps git+https://github.com/hddm-devs/kabuki

     |████████████████████████████████| 385 kB 7.7 MB/s 
  Created wheel for pymc: filename=pymc-2.3.8-cp37-cp37m-linux_x86_64.whl size=1361140 sha256=b98eade3bfcffc641c5676df0d2d4e0b81a552da154ba1f2f05a59d2b4e3b7a7
  Stored in directory: /root/.cache/pip/wheels/3b/9b/1d/2a2ddbe80519db2698745f0988ce50c36ad38aac886a4ee0ca
Successfully built pymc
  Cloning https://github.com/hddm-devs/hddm to /tmp/pip-req-build-zgba1np5
  Running command git clone -q https://github.com/hddm-devs/hddm /tmp/pip-req-build-zgba1np5
  Running command git submodule update --init --recursive -q
  Created wheel for HDDM: filename=HDDM-0.9.3-cp37-cp37m-linux_x86_64.whl size=3874730 sha256=350509b5a6fddb32ac309da52360b101f63424dc5af8ed3a763079f1a73d1c80
  Stored in directory: /tmp/pip-ephem-wheel-cache-t5ql_olf/wheels/12/ca/83/564eeefb14507aa5cef813e98168b986f6461da6bf04498e3a
Successfully built HDDM
  Cloning https://github.com/hddm-devs/kabuki to /tmp/pip-req-build-axweqow7
  Running command git clone -q https:/

In [ ]:
# MODULE IMPORTS ----
# warning settings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Data management
import pandas as pd
import numpy as np
import pickle

# Plotting
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

# Stats functionality
from statsmodels.distributions.empirical_distribution import ECDF

# HDDM
import hddm

In [ ]:
help(hddm.simulators.hddm_dataset_generators.simulator_h_c)

Help on function simulator_h_c in module hddm.simulators.hddm_dataset_generators:

simulator_h_c(data=None, n_subjects=10, n_trials_per_subject=100, model='ddm_vanilla', conditions=None, depends_on=None, regression_models=None, regression_covariates=None, group_only_regressors=True, group_only=['z'], fixed_at_default=None, p_outlier=0.0, outlier_max_t=10.0, **kwargs)
    Flexible simulator that allows specification of models very similar to the hddm model classes. Has two major modes. When data 
    
    is supplied the function generates synthetic versions of the provided data. If no data is provided, you can supply
    a varied of options to create complicated synthetic datasets from scratch.
    
    :Arguments:
        data: pd.DataFrame <default=None>
            Actual covariate dataset. If data is supplied its covariates are used instead of generated.
        n_subjects: int <default=5>
            Number of subjects in the datasets
        n_trials_per_subject: int <default=500

In [ ]:
## generate some artificial choice data with mediation variable



In [ ]:
# which HDDMnn models exist?
import pprint
mm = hddm.model_config.model_config.keys()
pprint.pprint(list(mm))
# for m in mm:
#   print('\n\n' + m)
#   pprint.pprint(hddm.model_config.model_config[m])

['ddm_vanilla',
 'full_ddm_vanilla',
 'ddm',
 'angle',
 'weibull',
 'levy',
 'full_ddm',
 'ornstein',
 'ddm_sdv',
 'ddm_par2',
 'ddm_par2_no_bias',
 'ddm_par2_angle_no_bias',
 'ddm_par2_weibull_no_bias',
 'ddm_seq2',
 'ddm_seq2_no_bias',
 'ddm_seq2_angle_no_bias',
 'ddm_seq2_weibull_no_bias',
 'ddm_mic2_adj',
 'ddm_mic2_adj_no_bias',
 'ddm_mic2_adj_angle_no_bias',
 'ddm_mic2_adj_weibull_no_bias',
 'race_no_bias_3',
 'race_no_bias_angle_3',
 'race_no_bias_4',
 'race_no_bias_angle_4',
 'lca_no_bias_3',
 'lca_no_bias_angle_3',
 'lca_no_bias_4',
 'lca_no_bias_angle_4',
 'weibull_cdf',
 'full_ddm2']


In [ ]:
# USE THE HDDMNN MODULE TO SIMULATE DATA
# see https://hddm.readthedocs.io/en/latest/lan_tutorial.html#section-5-regressors 

nmcmc = 1500
model = 'ddm' # start simple
n_subjects = 10
n_trials_per_subject = 500

# #         regression_covariates: dict <default={'covariate_name': {'type': 'categorical', 'range': (0, 4)}}>
#             Dictionary in dictionary. Specify the name of the covariate column as keys, and for each key supply the 'type' (categorical, continuous) and
#             'range' ((lower bound, upper bound)) of the covariate.
#             Example: {"covariate_name": {"type": "categorical", "range": (0, 4)}}

data, full_parameter_dict = hddm.simulators.hddm_dataset_generators.simulator_h_c(data, 
                                                                                  n_subjects = n_subjects,
                                                                                  n_trials_per_subject = n_trials_per_subject,
                                                                                  model = model,
                                                                                  p_outlier = 0.05,
                                                                                  conditions = None, 
                                                                                  depends_on = None,
                                                                                  regression_models = ['z ~ 1', 'v ~ 1 + stimulus + prevresp'], 
                                                                                  regression_covariates = {'stimulus': {'type': 'continuous', 'range': (0, 1)},
                                                                                                           'prevresp': {'type': 'continuous', 'range': (0, 1)}}, # need this to make initial covariate matrix from which to use dmatrix (patsy)
                                                                                  group_only_regressors = False,
                                                                                  group_only = None,
                                                                                  fixed_at_default = None)


In [ ]:
# Make HDDM Model to FIT

# Set up the regressor a regressor:
reg_model_v = {'model': 'v ~ 1 + stimulus + prevresp', 'link_func': lambda x: x}
reg_model_z = {'model': 'z ~ 1 + prevresp', 'link_func': lambda x: x}
reg_descr = [reg_model_z, reg_model_v]

hddmnn_model = hddm.HDDMnn(data,
                           model = model,
                           informative = False,
                           include = hddm.simulators.model_config[model]['hddm_include'],
                           p_outlier = 0.05,
                           is_group_model = True)
hddmnn_model.sample(nmcmc, burn = 100)

In [ ]:
# Caterpillar Plot: (Parameters recovered ok?)
hddm.plotting.plot_caterpillar(hddm_model = hddmnn_model,
                               ground_truth_parameter_dict = full_parameter_dict,
                               figsize = (8, 8),
                               columns = 3)
plt.show()